In [2]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 977457e8-9911-4d4d-b565-a2e10567eb9d.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 977457e8-9911-4d4d-b565-a2e10567eb9d.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 5.0


You are already connected to a glueetl session 977457e8-9911-4d4d-b565-a2e10567eb9d.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: None
Setting new worker type to: G.1X


You are already connected to a glueetl session 977457e8-9911-4d4d-b565-a2e10567eb9d.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: None
Setting new number of workers to: 5



In [3]:
#glue data catalog에서 테이블 읽어오기
dynamic_frame = glueContext.create_dynamic_frame.from_catalog(
    database="final_raw_data",  # Glue에서 정의된 데이터베이스 이름
    table_name="jeju_daily_weather",    # Glue에서 정의된 테이블 이름
    transformation_ctx="dynamic_frame"
)

# DynamicFrame을 DataFrame으로 변환
df = dynamic_frame.toDF()

# DataFrame에서 데이터를 조회
df.show(10)


+---------------+--------+------------------+-----------------+---------------------+------------------+----------------------+-------------+----------------+--------------------+----------------+-------------------------+----------+
|observatoryname|basedate|averagetemperature|lowesttemperature|lowesttemperaturetime|highesttemperature|highesttemperaturetime|dailyrainfall|maximumwindspeed|maximumwindspeedtime|averagewindspeed|maximumwindspeeddirection|searchdate|
+---------------+--------+------------------+-----------------+---------------------+------------------+----------------------+-------------+----------------+--------------------+----------------+-------------------------+----------+
|         마라도|20140101|              11.9|             10.7|                  718|              13.0|                  1451|          0.0|            14.8|                  21|             7.6|                    323.1|  20140101|
|       제주남원|20140101|              10.2|              5.2|        

In [10]:
from pyspark.sql.functions import col, to_date, date_format, format_number

df = df.withColumn("searchdate",
                   to_date(col("searchdate").cast("string"), "yyyyMMdd")
                  ).withColumn("basedate",
                               to_date(col("basedate").cast("string"), "yyyyMMdd")
                              )
df.show(10)
                   

+---------------+----------+------------------+-----------------+---------------------+------------------+----------------------+-------------+----------------+--------------------+----------------+-------------------------+----------+
|observatoryname|  basedate|averagetemperature|lowesttemperature|lowesttemperaturetime|highesttemperature|highesttemperaturetime|dailyrainfall|maximumwindspeed|maximumwindspeedtime|averagewindspeed|maximumwindspeeddirection|searchdate|
+---------------+----------+------------------+-----------------+---------------------+------------------+----------------------+-------------+----------------+--------------------+----------------+-------------------------+----------+
|         마라도|2014-01-01|              11.9|             10.7|                  718|              13.0|                  1451|          0.0|            14.8|                  21|             7.6|                    323.1|2014-01-01|
|       제주남원|2014-01-01|              10.2|              5.

In [ ]:
df.write.format("parquet").mode("overwrite").save("s3://airflow-test001/transformed_data/jeju-daily-weather/")